## Work
### 請結合前面的知識與程式碼，比較不同的 regularization 的組合對訓練的結果與影響：如 dropout, regularizers, batch-normalization 等

In [1]:
import os
import keras
import itertools
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [4]:
train, test = keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 6s 0us/step


In [5]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [6]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [7]:
from keras.regularizers import l2
def build_mlp(input_dim, num_neurons = [512, 256, 128], l2_ratio = 1e-2, dropout_rate = 0.5):
    """Code Here
    建立你的神經網路
    """
    input_layer = keras.layers.Input(input_dim)
    for i, n_units in enumerate(num_neurons):
      if i == 0:
        hidden_layer = keras.layers.Dense(units=n_units, activation='relu', kernel_regularizer=l2(l2_ratio))(input_layer)
        hidden_layer = keras.layers.Dropout(dropout_rate)(hidden_layer)
        hidden_layer = keras.layers.BatchNormalization()(hidden_layer)
      else:
        hidden_layer = keras.layers.Dense(units=n_units, activation='relu', kernel_regularizer=l2(l2_ratio))(hidden_layer)
        hidden_layer = keras.layers.Dropout(dropout_rate)(hidden_layer)
        hidden_layer = keras.layers.BatchNormalization()(hidden_layer)
    output_layer = keras.layers.Dense(units=10, activation='softmax')(hidden_layer)
    model = keras.models.Model(inputs = [input_layer], outputs = [output_layer])
    return model

In [10]:
"""Code Here
設定超參數
"""
learning_rate = 1e-3
l2_set = [1e-2, 1e-3]
dropout_set = [0.2, 0.5]
batch_set = [64, 512]
epochs = 50
momentum = 0.9

In [11]:
results = {}
"""Code Here
撰寫你的訓練流程並將結果用 dictionary 紀錄
"""
for batch_size in batch_set:
  for l2_ratio in l2_set:
    for dropout_rate in dropout_set:
      keras.backend.clear_session()
      print('batch_size:{}, l2_ratio:{}, dropout_rate:{}'.format(batch_size, l2_ratio, dropout_rate))
      model = build_mlp(input_dim=x_train.shape[1:], l2_ratio=l2_ratio, dropout_rate=dropout_rate)
      adam = keras.optimizers.Adam(learning_rate=learning_rate)
      model.compile(loss = 'categorical_crossentropy', metrics = ['accuracy'], optimizer = adam)
      model.fit(x_train, y_train, epochs = epochs, batch_size = batch_size, validation_data = (x_test, y_test), shuffle = True)

      train_loss = model.history.history['loss']
      val_loss = model.history.history['val_loss']
      train_acc = model.history.history['accuracy']
      val_acc = model.history.history['val_accuracy']
      name = 'batch_size:{}, l2_ratio:{}, dropout_rate:{}'.format(batch_size, l2_ratio, dropout_rate)
      results[name] = {'train_loss':train_loss, 
                       'val_loss':val_loss, 
                       'train_acc':train_acc, 
                       'val_acc':val_acc}

batch_size:64, l2_ratio:0.01, dropout_rate:0.2
Epoch 1/50
782/782 [==============================] - 4s 5ms/step - loss: 7.6420 - accuracy: 0.2348 - val_loss: 2.5245 - val_accuracy: 0.2336
Epoch 2/50
782/782 [==============================] - 3s 4ms/step - loss: 2.2403 - accuracy: 0.2820 - val_loss: 2.1885 - val_accuracy: 0.2401
Epoch 3/50
782/782 [==============================] - 3s 4ms/step - loss: 2.0621 - accuracy: 0.2890 - val_loss: 2.0168 - val_accuracy: 0.2859
Epoch 4/50
782/782 [==============================] - 3s 4ms/step - loss: 2.0331 - accuracy: 0.2831 - val_loss: 2.0114 - val_accuracy: 0.2803
Epoch 5/50
782/782 [==============================] - 3s 4ms/step - loss: 2.0385 - accuracy: 0.2684 - val_loss: 2.7088 - val_accuracy: 0.1317
Epoch 6/50
782/782 [==============================] - 3s 4ms/step - loss: 2.0116 - accuracy: 0.2842 - val_loss: 2.0727 - val_accuracy: 0.2358
Epoch 7/50
782/782 [==============================] - 3s 4ms/step - loss: 2.0266 - accuracy: 0.2723 -

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
"""Code Here
將結果繪出
"""